# Ollama + OpenAI + Python

## 1. モデル名を指定する

もし「phi3:mini」以外のモデルを取得した場合は、以下のセル内の値を変更してください。
この変数はノートブック全体のコードで使用されます。


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AIクライアントのセットアップ

通常、OpenAIクライアントはOpenAI.comやAzure OpenAIと共に使用され、大規模言語モデルと対話します。
しかし、Ollamaでも使用可能です。OllamaはOpenAI互換のエンドポイント「http://localhost:11434/v1」を提供しているためです。


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. チャットの応答を生成する

次に、OpenAI SDKを使用して会話の応答を生成します。このリクエストでは、猫についての俳句を生成する必要があります:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. プロンプトエンジニアリング

言語モデルに送信される最初のメッセージは「システムメッセージ」または「システムプロンプト」と呼ばれ、モデルの全体的な指示を設定します。
独自のカスタムシステムプロンプトを提供することで、言語モデルが異なる方法で出力を生成するように導くことができます。
以下の `SYSTEM_MESSAGE` を変更して、お気に入りの映画やテレビ番組のキャラクターのように答えるようにしたり、[Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) から他のシステムプロンプトのアイデアを得ることもできます。

システムメッセージをカスタマイズしたら、`USER_MESSAGE` に最初のユーザー質問を入力してください。


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. 少数例の提示

言語モデルを誘導するもう一つの方法は、「少数例」を提供することです。これは、質問と回答の例をいくつか示し、モデルがどのように応答すべきかを示すものです。

以下の例では、言語モデルにティーチングアシスタントのように振る舞わせることを試みています。ティーチングアシスタントが行いそうな質問と回答の例をいくつか提示し、その後、学生がしそうな質問をモデルに投げかけます。

まずは試してみてください。その後、新しいシナリオに合わせて `SYSTEM_MESSAGE`、`EXAMPLES`、`USER_MESSAGE` を変更してみましょう。


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. 検索強化型生成

RAG（検索強化型生成）は、特定の分野に関する質問に正確に答えるための手法です。まず知識ソースから関連情報を検索し、その情報に基づいて回答を生成します。

ハイブリッドカーに関するデータが記載されたローカルのCSVファイルを提供しています。以下のコードはCSVファイルを読み込み、ユーザーの質問に一致する情報を検索し、その情報に基づいて回答を生成します。この処理は、以前の例よりも時間がかかることに注意してください。モデルに送信するデータ量が増えるためです。もし回答がデータに基づいていないと感じた場合は、システムエンジニアリングを試すか、他のモデルを試してみてください。一般的に、RAGはより大きなモデルやSLMの微調整版を使用することで、より効果的になります。


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
